In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras

KeyboardInterrupt: 

In [ ]:
# Read well log data from CSV file
df = pd.read_csv("combined_freeman_2_3_4.csv")
df.head()

In [ ]:
# Split data into features (X) and target (y)
X = df.drop(columns=["PERM", 'well'])
y = df["PERM"]

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features using StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define ANN architecture
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]]),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train model
model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=16)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test)

# y_true: true y values, y_pred: predicted y values
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("R2 Score: {:.2f}".format(r2))


In [ ]:
# Reshape X_train and X_test into 2D images
n_samples_train, n_features_train = X_train.shape
n_samples_test, n_features_test = X_test.shape
image_shape = (1, int(n_features_train), 1)
X_train_img = np.reshape(X_train, (n_samples_train, *image_shape))
X_test_img = np.reshape(X_test, (n_samples_test, *image_shape))

# Define CNN architecture
model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=image_shape),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train model
model.fit(X_train_img, y_train, validation_split=0.2, epochs=50, batch_size=16)

In [ ]:
def calculate_porosity_from_density(density, matrix_density, fluid_density):
    """
    Calculates porosity from density logs given matrix and fluid density.
    Args:
        density (float or numpy array): density log value(s) in g/cc
        matrix_density (float): matrix density in g/cc
        fluid_density (float): fluid density in g/cc
    Returns:
        float or numpy array: porosity values
    """
    porosity = (matrix_density - density) / (matrix_density - fluid_density)
    return porosity


In [ ]:
d = calculate_porosity_from_density(2.105, 2.26, 1.1)
d